In [ ]:
from data_loader import load_and_prepare_data
from credit_models import modelValuation
from pytorch_tabnet.tab_model import TabNetClassifier
from tuner import tune_model

In [ ]:
(X_train, X_test, Y_train, Y_test), cat_idxs, cat_dims = load_and_prepare_data('tabnet')

# Get best hyperparameters from tuning
best_params = tune_model("tabnet", X_train, Y_train, n_trials=10, n_splits=2 , cat_idxs= cat_idxs , cat_dims= cat_dims)

# Prepare required fields from best_params
tabnet_params = {
    "n_d":              best_params["n_d"],
    "n_a":              best_params["n_a"],
    "n_steps":          best_params["n_steps"],
    "gamma":            best_params["gamma"],
    "lambda_sparse":    best_params["lambda_sparse"],
    "optimizer_params": {"lr": best_params["lr"]},
    "mask_type":        best_params["mask_type"],
    "cat_emb_dim":      best_params["cat_emb_dim"],
    "verbose":          0,
    "device_name":      "auto",
    "cat_idxs":         cat_idxs,
    "cat_dims":         cat_dims,
}


# Train final model
model = TabNetClassifier(**tabnet_params)
model.fit(
    X_train=X_train, y_train=Y_train,
    eval_set=[(X_test, Y_test)],
    eval_metric=["auc"],
    max_epochs=200,
    patience=20,
    batch_size=256,
    virtual_batch_size=128,
    weights=1
)

# Predict probabilities
y_proba = model.predict_proba(X_test)[:, 1]


In [ ]:
y_pred, chosen_thresh = modelValuation("TabNet", Y_test, y_proba, use_best_threshold=True)

In [ ]:
y_pred, chosen_thresh = modelValuation("TabNet", Y_test, y_proba, use_best_threshold=False)

In [ ]:
import joblib
joblib.dump(y_proba, "proba_tab.pkl")
model.save_model("tabnet_model.zip")
